In [1]:
from kloppy.domain.services.transformers.attribute import BodyPartTransformer, AngleToGoalTransformer, DistanceToGoalTransformer

from soccer_analytics.data.statsbomb import get_metadata, get_events
from soccer_analytics.data_split import split_by_time

In [2]:
competitions = get_metadata()

In [3]:
full_competitions = {
   "1. Bundesliga": ["2015/2016"],
#     "FA Women's Super League": ["2018/2019", "2019/2020", "2020/2021"],
#     "FIFA World Cup": ["2018", "2022"],
#     "Indian Super league": ["2021/2022"],
#     "La Liga": ["2015/2016"],
#     "Ligue 1": ["2015/2016"],
#     "Premier League": ["2015/2016"],
#     "Serie A": ["2015/2016"],
#     "UEFA Euro": ["2020"],
#     "UEFA Women's Euro": ["2022"],
#     "Women's World Cup": ["2019", "2023"]
}

In [4]:
matches = {}
for competition in competitions:
    print(competition.name, len(competition.seasons))
    if competition.name in full_competitions:
        matches[competition.name] = []
        for season in competition.seasons:
            print(f"    {season.name}: {len(season.matches)}")
            if season.name in full_competitions[competition.name]:
                matches[competition.name].extend(get_events(
                    season, event_types=["shot"]
                ))

1. Bundesliga 1
    2015/2016: 306
Champions League 18
Copa del Rey 3
FA Women's Super League 3
FIFA U20 World Cup 1
FIFA World Cup 8
Indian Super league 1
La Liga 18
Liga Profesional 2
Ligue 1 1
North American League 1
NWSL 1
Premier League 2
Serie A 2
UEFA Euro 1
UEFA Europa League 1
UEFA Women's Euro 1
Women's World Cup 2


In [5]:
all_matches = matches["1. Bundesliga"]

In [6]:
test = all_matches[0].to_df(
    "is_penalty",
    "is_first_time",
    "statsbomb_xg",
    "technique",
    "coordinates*",
    AngleToGoalTransformer(),
    DistanceToGoalTransformer(),
    BodyPartTransformer(encoding="one-hot"),
    is_goal=lambda event: event.result.is_success
)
test.head()

,is_penalty,is_first_time,statsbomb_xg,technique,coordinates_x,coordinates_y,angle_to_goal,distance_to_goal,is_body_part_right_foot,is_body_part_left_foot,is_body_part_head,is_body_part_both_hands,is_body_part_chest,is_body_part_left_hand,is_body_part_right_hand,is_body_part_drop_kick,is_body_part_keeper_arm,is_body_part_other,is_body_part_no_touch,is_goal
0,False,True,0.087901,Half Volley,108.05,31.15,53.476878,14.870272,False,True,False,False,False,False,False,False,False,False,False,False
1,False,True,0.160274,Volley,110.85,42.55,105.572544,9.498684,True,False,False,False,False,False,False,False,False,False,False,True
2,False,True,0.016036,Half Volley,117.85,29.05,11.108542,11.159077,False,True,False,False,False,False,False,False,False,False,False,False
3,False,None,0.527759,Normal,101.75,27.55,55.698513,22.092193,False,True,False,False,False,False,False,False,False,False,False,True
4,False,None,0.074020,Normal,109.25,26.45,38.427005,17.296387,True,False,False,False,False,False,False,False,False,False,False,False


In [7]:
test.technique.value_counts()

technique
Normal         19
Half Volley     3
Volley          1
Lob             1
Backheel        1
Name: count, dtype: int64

In [ ]:
test.dataset.metadata.orientation